In [198]:
from typing import Tuple, List
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

class DriftPointDetector:
    """
    ตรวจจับจุดเกิด concept drift ในข้อมูล time series ด้วยการใช้
    Kolmogorov-Smirnov test (KS test) บน sliding windows
    """
    def __init__(self, window_size: int = 50, threshold: float = 0.005):
        self.window_size = window_size
        self.threshold = threshold
        self.drift_points_: List[int] = []

    def detect(self, X: pd.DataFrame) -> List[int]:
        self.drift_points_ = []
        n = len(X)
        for i in range(self.window_size, n - self.window_size):
            window1 = X.iloc[i - self.window_size:i]
            window2 = X.iloc[i:i + self.window_size]

            drift_detected = False
            for col in X.columns:
                stat, p_value = ks_2samp(window1[col], window2[col])
                if p_value < self.threshold:
                    drift_detected = True
                    break

            if drift_detected:
                self.drift_points_.append(i)

        # กรองจุด drift ที่อยู่ใกล้กันเกินไป (ลด false positives)
        filtered_points = []
        last_point = -self.window_size
        for p in self.drift_points_:
            if p - last_point >= self.window_size:
                filtered_points.append(p)
                last_point = p
        self.drift_points_ = filtered_points

        return self.drift_points_

class AdaptiveFoldGenerator:
    """
    สร้าง train/test folds โดยแบ่งตาม drift points ที่ตรวจจับได้
    โดยจะข้ามช่วงที่สั้นเกินไป เพื่อให้แต่ละ fold มีขนาดพอเหมาะ
    """
    def __init__(self, min_fold_size: int = 50, test_ratio: float = 0.2):
        self.min_fold_size = min_fold_size
        self.test_ratio = test_ratio

    def split(self, X: pd.DataFrame, drift_points: List[int]) -> List[Tuple[np.ndarray, np.ndarray]]:
        folds = []
        points = [0] + drift_points + [len(X)]
        for i in range(len(points) - 1):
            start, end = points[i], points[i + 1]
            fold_length = end - start

            if fold_length < self.min_fold_size:
                continue  # ข้าม fold เล็ก ๆ

            split = int(start + (1 - self.test_ratio) * fold_length)
            train_idx = np.arange(start, split)
            test_idx = np.arange(split, end)

            folds.append((train_idx, test_idx))
        return folds

class DriftAdaptiveTimeSeriesCV:
    """
    ทำ cross-validation โดยใช้ fold ที่แบ่งตาม drift points
    ใช้ pipeline ที่กำหนดไว้ภายนอก (รับ parameter pipeline)
    """
    def __init__(self, pipeline: Pipeline):
        self.pipeline = pipeline

    def run(self, X: pd.DataFrame, y: pd.Series, drift_points: List[int]) -> Tuple[List[float], List[float]]:
        fold_gen = AdaptiveFoldGenerator()
        metrics_rmse, metrics_mae = [], []

        folds = fold_gen.split(X, drift_points)
        if not folds:
            print("Warning: No valid folds generated by AdaptiveFoldGenerator!")
            return [], []

        for i, (train_idx, test_idx) in enumerate(folds):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            # ใช้ pipeline ที่ส่งเข้ามา ไม่สร้างใหม่
            self.pipeline.fit(X_train, y_train)
            y_pred = self.pipeline.predict(X_test)

            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            print(f"[Adaptive Fold {i+1}] RMSE={rmse:.4f}, MAE={mae:.4f}")

            metrics_rmse.append(rmse)
            metrics_mae.append(mae)

        return metrics_rmse, metrics_mae

class BaselineTimeSeriesCV:
    """
    ทำ cross-validation แบบ TimeSeriesSplit ปกติ
    โดยใช้ pipeline ที่กำหนดไว้ภายนอก (รับ parameter pipeline)
    """
    def __init__(self, pipeline: Pipeline, n_splits: int = 5):
        self.pipeline = pipeline
        self.n_splits = n_splits

    def run(self, X: pd.DataFrame, y: pd.Series) -> Tuple[List[float], List[float]]:
        tscv = TimeSeriesSplit(n_splits=self.n_splits)
        metrics_rmse, metrics_mae = [], []

        for i, (train_idx, test_idx) in enumerate(tscv.split(X)):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            self.pipeline.fit(X_train, y_train)
            y_pred = self.pipeline.predict(X_test)

            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            print(f"[Baseline Fold {i+1}] RMSE={rmse:.4f}, MAE={mae:.4f}")

            metrics_rmse.append(rmse)
            metrics_mae.append(mae)

        return metrics_rmse, metrics_mae

# --------------------------------------------------------
# ตัวอย่างการใช้งาน
if __name__ == "__main__":
    np.random.seed(42)

    # โหลดข้อมูลหุ้น (ตัวอย่าง)
    df = pd.read_csv("nvidia_10yr_data.csv", parse_dates=["Date"])
    df = df.sort_values("Date")

    # Feature selection (เลือก column ที่ใช้เป็น X)
    X = df[['Open', 'High', 'Low', 'Volume']]
    y = df['Close']

    # สร้าง pipeline เดียวใช้ทั้ง Adaptive CV และ Baseline CV
    pipeline = Pipeline([
        ('scale', StandardScaler()),
        ('regressor', LinearRegression())
    ])

    # 1) Detect drift points
    detector = DriftPointDetector(window_size=300, threshold=0.05)
    drift_points = detector.detect(X)
    print(f"Detected drift points at indices: {drift_points}")

    # 2) Run Adaptive CV
    drift_cv = DriftAdaptiveTimeSeriesCV(pipeline)
    print("\n--- Drift-Aware Adaptive CV ---")
    drift_rmse, drift_mae = drift_cv.run(X, y, drift_points)

    # 3) Run Baseline TimeSeriesSplit CV
    baseline_cv = BaselineTimeSeriesCV(pipeline, n_splits=4)
    print("\n--- Baseline TimeSeriesSplit CV ---")
    base_rmse, base_mae = baseline_cv.run(X, y)


Detected drift points at indices: [871, 1544, 2053]

--- Drift-Aware Adaptive CV ---
[Adaptive Fold 1] RMSE=0.4364, MAE=0.1693
[Adaptive Fold 2] RMSE=0.3727, MAE=0.1609
[Adaptive Fold 3] RMSE=0.4395, MAE=0.1973
[Adaptive Fold 4] RMSE=0.2643, MAE=0.1447

--- Baseline TimeSeriesSplit CV ---
[Baseline Fold 1] RMSE=0.3712, MAE=0.1639
[Baseline Fold 2] RMSE=0.3755, MAE=0.1745
[Baseline Fold 3] RMSE=0.3825, MAE=0.1662
[Baseline Fold 4] RMSE=0.4611, MAE=0.1746
